In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -q catboost

In [1]:
import pandas as pd
from sklearn import preprocessing
import pandas as pd
import joblib
from sklearn.preprocessing import OrdinalEncoder
import os
import lightgbm as lgb 
import numpy as np
from catboost import CatBoostClassifier

In [2]:
root='/content/gdrive/MyDrive/Colab NotebooksTH/proriv2022/task1/dataset'
path_test_norm=os.path.join(root,'test_dataset_test_norm.csv')
path_ckpt='/content/gdrive/MyDrive/Colab NotebooksTH/proriv2022/task1/solution/best/catboost_55.ckpt'
path_subm_v55='/content/gdrive/MyDrive/Colab NotebooksTH/proriv2022/task1/solution/best/my_submission_55.csv'
path_chkpts=os.path.join('/content/gdrive/MyDrive/Colab NotebooksTH/proriv2022/task1/solution/checkpoints')
enc=joblib.load(os.path.join(path_chkpts,'labelEncoder.joblib'))
enc.categories_

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


[array([ 0,  1,  3,  4,  5, 64])]

In [3]:
df_test=pd.read_csv(path_test_norm)
df_test.head(3)

,Easting,Northing,Height,Reflectance
0,1.606577,-1.439124,-0.062096,-0.765618
1,-1.758589,1.229095,-0.611865,-0.101560
2,-0.669053,0.870488,-0.069409,0.060027


In [4]:
subm_v55=pd.read_csv(path_subm_v55)
subm_v55.head()

,id,Class
0,1836965,0
1,3659389,3
2,5433033,0
3,1168709,3
4,5821779,3


In [5]:
for df in [df_test]:
    df['log_Reflectance_10']=np.log(df.Reflectance+10)  
    #####################
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2)+np.power(df.Height,2))
    df['Phi']=(df.Easting+df.Northing+df.Height)/denominator  
    df['ENH']=df.Easting*df.Northing/df.Height  
    ####################################################
    df['cos_E']=np.cos(df.Easting)
    df['cos_N']=np.cos(df.Northing)
    df['cos_H']=np.cos(df.Height)
    #####################################################
    df['sin_E']=np.sin(df.Easting)
    df['sin_N']=np.sin(df.Northing)
    df['sin_H']=np.sin(df.Height)
    #####################################################
    #denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Height,2))
    #df['Phi_EH']=(df.Easting+df.Height)/denominator  
    #denominator=np.sqrt(np.power(df.Northing,2)+np.power(df.Height,2))
    #df['Phi_NH']=(df.Northing+df.Height)/denominator  
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2))
    df['Phi_EN']=(df.Easting+df.Northing)/denominator  

    ###################
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2)+np.power(df.Height,2))
    df['Phi_abs']=(abs(df.Easting)+abs(df.Northing)+abs(df.Height))/denominator  
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2))
    df['Phi_EN_abs']=(abs(df.Easting)+abs(df.Northing))/denominator  


    #####################
    denominator=np.sqrt(np.sqrt(abs(df.Easting))+np.sqrt(abs(df.Northing))+np.sqrt(abs(df.Height)))
    df['Phi_sqrt']=(df.Easting+df.Northing+df.Height)/denominator  
    df['Phi_ccs_sqrt']=(df.cos_E+df.cos_N+df.sin_H)/denominator  #one sine?
    df['Phi_csc_sqrt']=(df.cos_E+df.sin_N+df.cos_H)/denominator  
    df['Phi_scc_sqrt']=(df.sin_E+df.cos_N+df.cos_H)/denominator  

    ########################

    df.drop(['Reflectance','Easting','Northing','sin_H'],axis=1,inplace=True)

In [6]:
model_catboost_v55 = CatBoostClassifier()     
model_catboost_v55.load_model(path_ckpt)
y_pred_v55_df_test=model_catboost_v55.predict_proba(df_test)

In [10]:
subm_v55['check']=enc.inverse_transform(np.argmax(y_pred_v55_df_test,axis=1).reshape(len(y_pred_v55_df_test),-1))
subm_v55.head()

,id,Class,check
0,1836965,0,0
1,3659389,3,3
2,5433033,0,0
3,1168709,3,3
4,5821779,3,3


In [12]:
(subm_v55.Class-subm_v55.check).sum()

0

Все ок! модель работает